# CUDA and volume display example

Rotate a 3D volume and maximum-intensity project (MIP) to 2D

In [1]:
#imports & defs
from __future__ import print_function, division

#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as ipyw
from ipywidgets import interact

from caspyr.cuda import CudaFunc3D, cu3D, InC, OutC
from caspyr.plotting import volshow
import numpy as np
from scipy import ndimage
from tqdm import tqdm, trange

def imshow(im):
    plt.imshow(im, origin="lower", cmap="Greys")
    plt.axis("off")
    plt.draw()

## Generate data

In [2]:
dat = np.random.rand(128, 128, 128)
dat[60:, 10:14, 50:60] += 0.2
dat[:90, 42:48, 100:110] += 0.2
dat[60:80, 32:38, 100:110] += 0.2
dat[:90, 100:110, 40:50] += 0.2
volshow([dat]);

<Figure size 432x288 with 0 Axes>

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT02NCwgZGVzY3JpcHRpb249dSd6JywgbWF4PTEyNyksIE91dHB1dCgpKSwgX2RvbV9jbGFzc2VzPSh1J3dpZGdldC1pbnRlcmHigKY=


## CPU code

In [3]:
figMipRot = plt.figure()
def mipRot(angle):
    plt.figure(figMipRot.number)
    imshow(ndimage.interpolation.rotate(dat, angle, axes=(1, 2), reshape=False, order=0).max(axis=1))

ipyw.interact(mipRot, angle=ipyw.IntSlider(min=0, max=359, step=1, value=0));

<Figure size 432x288 with 0 Axes>

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2FuZ2xlJywgbWF4PTM1OSksIE91dHB1dCgpKSwgX2RvbV9jbGFzc2VzPSh1J3dpZGdldC1pbnTigKY=


## GPU code

In [4]:
MIP_ROT_CU = """
( DTYPE dst[N_DIM_Z][N_DIM_X]
, const DTYPE src[N_DIM_Z][N_DIM_Y][N_DIM_X]
, DTYPE ang
){
    SIZE_T x = blockIdx.x * B_DIM_X + threadIdx.x; if(x >= N_DIM_X) return;
    SIZE_T y = blockIdx.y * B_DIM_Y + threadIdx.y; if(y >= N_DIM_Y) return;
    SIZE_T z = blockIdx.z * B_DIM_Z + threadIdx.z; if(z >= N_DIM_Z) return;

    DTYPE xMid = x - N_DIM_X / 2.0;
    DTYPE yMid = y - N_DIM_Y / 2.0;
    ang *= -DTYPE(3.1415926535897932384626433832795 / 180);
    DTYPE s = sin(ang);
    DTYPE c = cos(ang);

    int x_ = round(N_DIM_X / 2.0 + xMid * c + yMid * s); if(0 > x_ || x_ >= N_DIM_X) return;
    int y_ = round(N_DIM_Y / 2.0 - xMid * s + yMid * c); if(0 > y_ || y_ >= N_DIM_Y) return;

    dst[z][x] = max(dst[z][x], src[z][y_][x_]);
}"""

### OOP

In [5]:
class MIPRot(CudaFunc3D):
    def __init__(self, shape, block_dim, dtype):
        super(MIPRot, self).__init__(shape[::-1], block_dim=block_dim, dtype=dtype)
        self.build(MIP_ROT_CU)
        self._res = np.empty((shape[0], shape[2]), dtype=dtype)  # pre-allocate

    def __call__(self, dat, angle):
        self._res.fill(0)
        super(MIPRot, self).__call__(self.OutC(self._res), self.InC(dat), self.dtype(angle))
        return self._res  # .copy()
mipRotKnl = MIPRot(dat.shape, [32, 1, 32], dat.dtype)  # pre-build kernel
datGPU = InC(dat)  # pre-wrap

figMipRotGpu = plt.figure()
def mipRotGpu(angle):
    plt.figure(figMipRotGpu.number)
    # mipRotKnl = MIPRot(dat.shape, [32, 1, 32], dat.dtype)
    res = mipRotKnl(datGPU, angle)
    imshow(res)

ipyw.interact(mipRotGpu, angle=ipyw.IntSlider(min=0, max=359, step=1, value=0));

<Figure size 432x288 with 0 Axes>

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2FuZ2xlJywgbWF4PTM1OSksIE91dHB1dCgpKSwgX2RvbV9jbGFzc2VzPSh1J3dpZGdldC1pbnTigKY=


## GPU functional

- Quicker to write (especially for constant launch parameters)
- More CUDA-like interface
```cpp
// CUDA
#define DTYPE float
#define SIZE_T unsigned
__global__ void func(DTYPE dst[], const DTYPE src[], SIZE_T N){ ... }
func<<<shape, blocks>>>(d, s, N);
```
```python
# Python
func_knl = cu3D("""(DTYPE dst[N_DIM_X], const DTYPE src[N_DIM_X]){ ... }""")
func = func_knl(shape, blocks, dtype=np.float32, size_t=np.uint32)
func(d, s)
```
- Less extensible

In [6]:
MIPRotCode = cu3D(MIP_ROT_CU)
_mipRotKnl2 = MIPRotCode(dat.shape[::-1], block_dim=[32, 1, 32], dtype=dat.dtype)  # pre-build kernel
_mipRotKnl2Res = np.empty((dat.shape[0], dat.shape[2]), dtype=dat.dtype)  # pre-allocate
def mipRotKnl2(dat, angle):
    # _mipRotKnl2Res = np.zeros((dat.shape[0], dat.shape[2]), dtype=dat.dtype)
    _mipRotKnl2Res.fill(0)
    # _mipRotKnl2 = MIPRotCode(dat.shape[::-1], block_dim=[32, 1, 32], dtype=dat.dtype)
    _mipRotKnl2(OutC(_mipRotKnl2Res), InC(dat), _mipRotKnl2Res.dtype.type(angle))
    return _mipRotKnl2Res

## Perfomance test

In [7]:
[ndimage.interpolation.rotate(dat, i, axes=(1, 2), reshape=False, order=0).max(axis=1)
                       for i in trange(0, 360, 10,desc="CPU                 ")]
[mipRotKnl (dat   , i) for i in trange(0, 360, 1, desc="GPU (OOP)           ")]
[mipRotKnl2(dat   , i) for i in trange(0, 360, 1, desc="GPU (functional)    ")];
[mipRotKnl (datGPU, i) for i in trange(0, 360, 1, desc="GPU (OOP)        dev")]
[mipRotKnl2(datGPU, i) for i in trange(0, 360, 1, desc="GPU (functional) dev")];

GPU (functional) dev: 100%|██████████| 360/360 [00:02<00:00, 158.60it/s]
